In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torchvision import models
from copy import deepcopy
import os

from nn_extrapolation import AcceleratedSGD
from nn_utils import *

In [3]:
trainer = Trainer(
    device="cuda:2",
    loss_fn=nn.NLLLoss(reduction="mean"),
    val_loss_fn=nn.NLLLoss(reduction="sum"),
)

In [4]:
dl = load_dataset(
    dataset="CIFAR10",
    root=os.path.join("/tmp", os.environ["USER"], "CIFAR"),
    augmentation=transforms.RandomAffine(10, scale=(0.9, 1.1), translate=(0.2, 0.2)),
    validation_split=0.2,
    batch_size=128,
    num_workers=10,
)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.LogSoftmax(-1)
)
initial_state = deepcopy(model.state_dict())
model.to(trainer.device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## $ k = 5 $

In [6]:
trainer.validation(model, dl["valid"])

(0.0922, 2.3062081256866453)

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-1, k=5, momentum=0.9, weight_decay=1e-5, lambda_=1e-8)
logger = Logger("resnet_log_augmentation-early_stopping-k=5.txt.no_resizing")

In [ ]:
torch.manual_seed(2020)
max_epochs = 300
early_stopping = EarlyStopping(15)

for epoch in range(max_epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    if early_stopping.should_stop(val_loss):
        break

100%|██████████| 313/313 [00:16<00:00, 19.49it/s, loss=2.5121]
Epoch 1 | Training loss: 2.5121, validation accuracy: 0.2984, validation loss: 1.9202
100%|██████████| 313/313 [00:16<00:00, 19.25it/s, loss=1.8013]
Epoch 2 | Training loss: 1.8013, validation accuracy: 0.3523, validation loss: 1.7481
100%|██████████| 313/313 [00:16<00:00, 19.36it/s, loss=1.6384]
Epoch 3 | Training loss: 1.6384, validation accuracy: 0.4105, validation loss: 1.5933
100%|██████████| 313/313 [00:16<00:00, 18.59it/s, loss=1.5383]
Epoch 4 | Training loss: 1.5383, validation accuracy: 0.4787, validation loss: 1.4367
100%|██████████| 313/313 [00:17<00:00, 18.12it/s, loss=1.4619]
Epoch 5 | Training loss: 1.4619, validation accuracy: 0.4906, validation loss: 1.4018
100%|██████████| 313/313 [00:16<00:00, 18.44it/s, loss=1.3885]
Epoch 6 | Training loss: 1.3885, validation accuracy: 0.5472, validation loss: 1.2321
100%|██████████| 313/313 [00:17<00:00, 18.38it/s, loss=1.3101]
Epoch 7 | Training loss: 1.3101, validation

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.to(trainer.device)

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.to(trainer.device)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.param_groups[0]["lr"] = 1e-2

In [ ]:
max_epochs = 300
early_stopping = EarlyStopping(15)

for epoch in range(max_epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    if early_stopping.should_stop(val_loss):
        break

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.to(trainer.device)

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.to(trainer.device)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.param_groups[0]["lr"] = 1e-3

In [ ]:
max_epochs = 300
early_stopping = EarlyStopping(15)

for epoch in range(max_epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    if early_stopping.should_stop(val_loss):
        break

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.to(trainer.device)

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.to(trainer.device)

## $ k = 10 $

In [ ]:
model.load_state_dict(initial_state)
model.to(trainer.device)
trainer.validation(model, dl["valid"])

In [ ]:
optimizer = AcceleratedSGD(model.parameters(), 1e-1, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-8)
logger = Logger("resnet_log_augmentation-early_stopping-k=10.txt.no_resizing")

In [ ]:
torch.manual_seed(2020)
max_epochs = 300
early_stopping = EarlyStopping(15)

for epoch in range(max_epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    if early_stopping.should_stop(val_loss):
        break

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.cuda()

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.cuda()

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.param_groups[0]["lr"] = 1e-2

In [ ]:
max_epochs = 300
early_stopping = EarlyStopping(15)

for epoch in range(max_epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    if early_stopping.should_stop(val_loss):
        break

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.cuda()

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.cuda()

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.param_groups[0]["lr"] = 1e-3

In [ ]:
max_epochs = 300
early_stopping = EarlyStopping(15)

for epoch in range(max_epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    if early_stopping.should_stop(val_loss):
        break

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.cuda()

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log(optimizer.param_groups[0]["method"])
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)
model_acc.cpu()
model.cuda()

In [ ]:
exit